# RFM Analysis

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_excel("D:\document\data_science\Mini Project dan final Project\RFM Analysis.xlsx")

In [4]:
df.shape

(2155, 4)

In [5]:
df.head()

,OrderID,OrderDate,CustomerID,Monetory
0,10248,1996-07-04 00:00:00.000,VINET,168.0
1,10248,1996-07-04 00:00:00.000,VINET,98.0
2,10248,1996-07-04 00:00:00.000,VINET,174.0
3,10249,1996-07-05 00:00:00.000,TOMSP,167.4
4,10249,1996-07-05 00:00:00.000,TOMSP,1696.0


In [12]:
df = df[df["OrderDate"] < "1998-01-01"]

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1464 entries, 0 to 1463
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OrderID     1464 non-null   int64         
 1   OrderDate   1464 non-null   datetime64[ns]
 2   CustomerID  1464 non-null   object        
 3   Monetory    1464 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 57.2+ KB


In [18]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

# FRM table

In [7]:
import datetime as dt
NOW = dt.datetime(1998,1,1)

In [8]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])

In [22]:
rfmtable = df.groupby('CustomerID').agg({'OrderDate': lambda x: (NOW - x.max()).days,
                                        'OrderID': lambda x: len(x),
                                        'Monetory': lambda x: x.sum()})

rfmtable["OrderDate"] = rfmtable['OrderDate'].astype(int)
rfmtable.rename(columns={'OrderDate': 'recency',
                        'OrderID': 'requency'})

,recency,requency,Monetory
CustomerID,,,
ALFKI,80,6,2294.00
ANATR,34,6,888.55
ANTON,98,15,6855.35
AROUT,8,23,7968.00
BERGS,16,36,18857.60
...,...,...,...
WARTH,15,35,16317.10
WELLI,2,13,5235.70
WHITC,49,27,13794.55


# RFM quartiles